In [ ]:
import os
import sys 

while "src" not in os.listdir(): os.chdir("..")

if "./src" not in sys.path: sys.path.append("./src")

In [ ]:
from os import getenv

from uuid import uuid4

from langchain_openai import ChatOpenAI

from dotenv import load_dotenv

load_dotenv()

llm = ChatOpenAI(
  api_key=getenv("OPENAI_API_KEY"),
  base_url="https://openrouter.ai/api/v1",
  model="gpt-4o-mini",
)

In [ ]:
import os
import openai

from typing import TypedDict

from langgraph.checkpoint.memory import InMemorySaver

from langgraph.graph import StateGraph, START, END
from langgraph.types import interrupt, Command

openai.api_key = os.environ.get("OPENAI_API_KEY")

class SubAgentState(TypedDict):
    user_input: str = ""
    ai_output: str = ""
    complete:bool = False 

def input_node(state: SubAgentState):
    """
    Node 1: interrupt for input and store it in StateGraph['user_input'].
    (In a real langgraph node this would be an interrupt node; here we use Python input().)
    """
    if "complete" not in state: state["complete"] = False
    prompt = interrupt("Enter prompt (or 'exit' to exit): ").strip()
    if isinstance(prompt, dict):
        state = prompt["state"]
        prompt = prompt["prompt"]
    state['user_input'] = prompt 
    state["complete"] = state["user_input"].strip().lower() == "exit"
    return state 

def llm_node(state: SubAgentState):
    """
    Node 2: call LLM using the stored input in StateGraph['user_input'].
    After running, this node returns control back to the input node (the loop below).
    """
    ai_message = llm.invoke(state["user_input"])
    state["ai_output"] = ai_message.content
    print (state["ai_output"])
    return state

def quit_or_not(state: SubAgentState):
    return "llm_node" if not state["complete"] else END

builder = StateGraph(SubAgentState)
builder.add_node("input_node", input_node)
builder.add_node("llm_node", llm_node)

builder.add_edge(START, "input_node")
builder.add_conditional_edges("input_node", quit_or_not, ["llm_node", END])
builder.add_edge("llm_node", "input_node")

config = {"configurable": {"thread_id": str(uuid4())}}
sa = builder.compile(checkpointer=InMemorySaver())

# interrupted = False 
# result = {"user_input": "", "ai_output": "", "complete": False}
# while not result["complete"]: 
#     if interrupted:
#         result = sa.invoke(Command(resume=input("Prompt: ")), config=config)
#     else:
#         result = sa.invoke(input=result, config=config)
#     interrupted = "__interrupt__" in result
        
    

In [ ]:
from uuid import uuid4
from typing import Annotated
from langchain_core.tools import InjectedToolCallId
# from langchain_core.tools import tool
from langchain.tools import ToolRuntime, tool
from langchain_core.messages import HumanMessage, AnyMessage, SystemMessage, ToolMessage
from langgraph.types import Command, interrupt
from langgraph.graph import add_messages
from typing import Literal

@tool(
    "chatting_agent_tool",
    description="A tool for interacting with a specialized agent."
)
def agent_tkt_tool(runtime: ToolRuntime) -> str:
    result = {"user_input": "", "ai_output": "", "complete": False}
    _config = config
    while not result["complete"]: 
        interrupted = runtime.state["interrupted"]
        if interrupted:
            resume = interrupt("Prompt: ")
            result = sa.invoke(Command(resume=resume), config=_config)
        else:
            result = sa.invoke(input=result, config=_config)
        interrupted = "__interrupt__" in result
        

class MasterState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]
    interrupted: bool = False

tools = [agent_tkt_tool] 
tool_name_to_executables = {tool.name: tool for tool in tools}
llm_with_tools = llm.bind_tools(tools)

def llm_call(state: MasterState) -> MasterState:

    _input = [SystemMessage("You are a routing agent. Route the user to chat with the specialized agent.")] + \
                state["messages"]

    ai_message = llm_with_tools.invoke(
            input=_input
        )
    state["messages"] = [ai_message]
    return state

def check_tool_call(state: MasterState) -> Literal["tool_executor", "END"]:
    last_message = state["messages"][-1]
    if last_message.tool_calls:
        return "tool_executor"
    else:
        return END

def tool_executor(state: MasterState):
    result = []
    for tc in state["messages"][-1].tool_calls:
        _id, _name, _args = tc["id"], tc["name"], tc["args"]
        _args.update(state)
        tool_response = tool_name_to_executables[_name].invoke(_args)
        result.append(ToolMessage(content=tool_response, tool_call_id=_id))
    state["messages"].extend(result)
    return state

builder = StateGraph(MasterState)
builder.add_node("llm_call", llm_call)
builder.add_node("tool_executor", tool_executor)
builder.add_edge(START, "llm_call")
builder.add_conditional_edges("llm_call", check_tool_call, ["tool_executor", END])
builder.add_edge("tool_executor", "llm_call")

ma = builder.compile(checkpointer=InMemorySaver())

blar = True
config = {"configurable": {"thread_id": str(uuid4())}}
interrupted = False 
result = {"messages": [], "interrupted": False}
inputs = ["testing", "exit"]
input_iter = iter(inputs)
while blar: 
    if interrupted:
        prompt = next(input_iter) # input("Prompt:")
        resume = {
            "prompt": prompt, 
            "config": config,
            "state": result,
        }
        result = ma.invoke(Command(resume=resume, update={"interrupted" : True}), config=config)
    else:
        result = ma.invoke(input=result, config=config)
    interrupted = "__interrupt__" in result
